In [1]:
import os
import pandas as pd
import glob
import json
from IPython.display import display

In [2]:
# Data paths
data_folder_path = '/mnt/4TBSSD/nvtu/LSC23_Data_Mount'
metadata_path = os.path.join(data_folder_path, 'lsc22_metadata.csv')
visual_concept_path = os.path.join(data_folder_path, 'lsc22_visual_concepts.csv')
gps_data_path = os.path.join(data_folder_path, 'vaisl_gps.csv')

# Load data
metadata = pd.read_csv(metadata_path, low_memory=False)
vc_data = pd.read_csv(visual_concept_path, low_memory=False)
gps_data = pd.read_csv(gps_data_path, low_memory=False)

In [8]:
print(metadata.columns)
print(vc_data.columns)
print(gps_data.columns)
display(metadata.head())
display(vc_data.head())
display(gps_data.head())
display(metadata.head())

Index(['Unnamed: 0', 'minute_id', 'utc_time', 'local_time', 'latitude',
       'longitude', 'altitude', 'semantic_name', 'time_zone',
       'heart_rate(bpm)', 'heart_rate_conf', 'calories', 'distance',
       'artist name', 'song name', 'album name', 'sleep_level', 'awake',
       'minutesToFallAsleep', 'minutesAsleep', 'minutesAwake',
       'minutesAfterWakeup', 'timeInBed', 'sleep_efficiency', 'ImageID'],
      dtype='object')
Index(['Unnamed: 0', 'ImageID', 'Tags', 'OCR', 'Caption', 'CaptionScore'], dtype='object')
Index(['ImageID', 'minute_id', 'stop', 'new_lat', 'new_lng', 'semantic_name',
       'foursquare_id', 'original_name', 'categories', 'parent', 'movement',
       'movement_prob', 'city', 'country', 'new_timezone'],
      dtype='object')


,Unnamed: 0,minute_id,utc_time,local_time,latitude,longitude,altitude,semantic_name,time_zone,heart_rate(bpm),...,album name,sleep_level,awake,minutesToFallAsleep,minutesAsleep,minutesAwake,minutesAfterWakeup,timeInBed,sleep_efficiency,ImageID
637,637,20190101_1037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,116.0,...,NaN,NaN,NaN,0.0,299.0,52.0,5.0,351.0,90.0,"['20190101_103717_000.jpg', '20190101_103749_0..."
638,638,20190101_1038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,114.0,...,NaN,NaN,NaN,0.0,299.0,52.0,5.0,351.0,90.0,"['20190101_103821_000.jpg', '20190101_103853_0..."
639,639,20190101_1039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,119.0,...,NaN,NaN,NaN,0.0,299.0,52.0,5.0,351.0,90.0,"['20190101_103925_000.jpg', '20190101_103957_0..."
640,640,20190101_1040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,119.0,...,NaN,NaN,NaN,0.0,299.0,52.0,5.0,351.0,90.0,['20190101_104029_000.jpg']
641,641,20190101_1041,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.0,...,NaN,NaN,NaN,0.0,299.0,52.0,5.0,351.0,90.0,"['20190101_104101_000.jpg', '20190101_104133_0..."


,Unnamed: 0,ImageID,Tags,OCR,Caption,CaptionScore
0,0,20000101_000113_000.jpg,"wall,bathroom,indoor,toilet,tile,tiled",NaN,a mirror and a shelf,0.215212
1,1,20000101_000145_000.jpg,"text,wall,indoor",NaN,a room with a tv and a window,0.440993
2,2,20000101_000217_000.jpg,"text,wall,indoor",NaN,a wall with a light on it,0.465437
3,3,20000101_000249_000.jpg,text,NaN,a wall with a light on it,0.464277
4,4,20000101_000321_000.jpg,"text,wall,indoor",NaN,a wall with a light on it,0.456158


,ImageID,minute_id,stop,new_lat,new_lng,semantic_name,foursquare_id,original_name,categories,parent,movement,movement_prob,city,country,new_timezone
0,20190101_103717_000.jpg,20190101_1037,True,53.38998,-6.14576,HOME,NaN,NaN,NaN,NaN,Inside,0.981373,"Dublin, Ireland, Leinster",Ireland,Europe/Dublin
1,20190101_103749_000.jpg,20190101_1037,True,53.38998,-6.14576,HOME,NaN,NaN,NaN,NaN,Inside,0.997463,"Dublin, Ireland, Leinster",Ireland,Europe/Dublin
2,20190101_103821_000.jpg,20190101_1038,True,53.38998,-6.14576,HOME,NaN,NaN,NaN,NaN,Inside,0.875969,"Dublin, Ireland, Leinster",Ireland,Europe/Dublin
3,20190101_103853_000.jpg,20190101_1038,True,53.38998,-6.14576,HOME,NaN,NaN,NaN,NaN,Inside,0.998937,"Dublin, Ireland, Leinster",Ireland,Europe/Dublin
4,20190101_103925_000.jpg,20190101_1039,True,53.38998,-6.14576,HOME,NaN,NaN,NaN,NaN,Inside,0.756499,"Dublin, Ireland, Leinster",Ireland,Europe/Dublin


In [9]:
metadata = metadata.drop(metadata[metadata['ImageID'].isnull()].index, axis=0)
metadata = metadata.rename({
    'ImageID': 'ImageIDs'
}, axis=1)

# Split the ImageIDs into ImageID
metadata = (
 metadata.assign(ImageID=metadata['ImageIDs'].apply(eval))
   .explode('ImageID')
   .reset_index(drop=True)
)

# Drop unused columns
metadata = metadata.drop([
    'ImageIDs', 'latitude',
       'longitude', 'altitude', 'minute_id', 'semantic_name',
], axis=1)
vc_data = vc_data.drop('Unnamed: 0', axis=1)
metadata = metadata.drop('Unnamed: 0', axis=1)
display(metadata)

,utc_time,local_time,time_zone,heart_rate(bpm),heart_rate_conf,calories,distance,artist name,song name,album name,sleep_level,awake,minutesToFallAsleep,minutesAsleep,minutesAwake,minutesAfterWakeup,timeInBed,sleep_efficiency,ImageID
0,NaN,NaN,NaN,116.0,2.0,5.94,2070.0,NaN,NaN,NaN,NaN,NaN,0.0,299.0,52.0,5.0,351.0,90.0,20190101_103717_000.jpg
1,NaN,NaN,NaN,116.0,2.0,5.94,2070.0,NaN,NaN,NaN,NaN,NaN,0.0,299.0,52.0,5.0,351.0,90.0,20190101_103749_000.jpg
2,NaN,NaN,NaN,114.0,2.0,5.23,2930.0,NaN,NaN,NaN,NaN,NaN,0.0,299.0,52.0,5.0,351.0,90.0,20190101_103821_000.jpg
3,NaN,NaN,NaN,114.0,2.0,5.23,2930.0,NaN,NaN,NaN,NaN,NaN,0.0,299.0,52.0,5.0,351.0,90.0,20190101_103853_000.jpg
4,NaN,NaN,NaN,119.0,1.0,5.70,350.0,NaN,NaN,NaN,NaN,NaN,0.0,299.0,52.0,5.0,351.0,90.0,20190101_103925_000.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
723324,2020-06-30 21:18:13,2020-06-30 22:18:13,Europe/Dublin,75.0,3.0,1.40,NaN,NaN,NaN,NaN,NaN,NaN,0.0,370.0,18.0,0.0,388.0,95.0,20200630_211803_000.jpg
723325,2020-06-30 21:19:31,2020-06-30 22:19:31,Europe/Dublin,76.0,3.0,1.40,NaN,NaN,NaN,NaN,NaN,NaN,0.0,370.0,18.0,0.0,388.0,95.0,20200630_211908_000.jpg
723326,2020-06-30 21:21:09,2020-06-30 22:21:09,Europe/Dublin,76.0,3.0,1.40,0.0,NaN,NaN,NaN,NaN,NaN,0.0,370.0,18.0,0.0,388.0,95.0,20200630_212119_000.jpg
723327,2020-06-30 21:21:09,2020-06-30 22:21:09,Europe/Dublin,76.0,3.0,1.40,0.0,NaN,NaN,NaN,NaN,NaN,0.0,370.0,18.0,0.0,388.0,95.0,20200630_212152_000.jpg


In [10]:
# Merge based on ImageID
overall_df = pd.merge(vc_data, metadata, on='ImageID', how='outer')
overall_df = pd.merge(overall_df, gps_data, on='ImageID', how='outer')
overall_df['ImageID'] = overall_df.apply(lambda x: x['ImageID'].split('.jpg')[0], axis=1)

In [11]:
list(overall_df.columns)

['ImageID',
 'Tags',
 'OCR',
 'Caption',
 'CaptionScore',
 'utc_time',
 'local_time',
 'time_zone',
 'heart_rate(bpm)',
 'heart_rate_conf',
 'calories',
 'distance',
 'artist name',
 'song name',
 'album name',
 'sleep_level',
 'awake',
 'minutesToFallAsleep',
 'minutesAsleep',
 'minutesAwake',
 'minutesAfterWakeup',
 'timeInBed',
 'sleep_efficiency',
 'minute_id',
 'stop',
 'new_lat',
 'new_lng',
 'semantic_name',
 'foursquare_id',
 'original_name',
 'categories',
 'parent',
 'movement',
 'movement_prob',
 'city',
 'country',
 'new_timezone']

In [12]:
overall_df = overall_df.set_index('ImageID')
json_dict = overall_df.to_json(orient="index")
parsed = json.loads(json_dict)

In [13]:
parsed

{'20000101_000113_000': {'Tags': 'wall,bathroom,indoor,toilet,tile,tiled',
  'OCR': None,
  'Caption': 'a mirror and a shelf',
  'CaptionScore': 0.2152121663,
  'utc_time': None,
  'local_time': None,
  'time_zone': None,
  'heart_rate(bpm)': None,
  'heart_rate_conf': None,
  'calories': None,
  'distance': None,
  'artist name': None,
  'song name': None,
  'album name': None,
  'sleep_level': None,
  'awake': None,
  'minutesToFallAsleep': None,
  'minutesAsleep': None,
  'minutesAwake': None,
  'minutesAfterWakeup': None,
  'timeInBed': None,
  'sleep_efficiency': None,
  'minute_id': None,
  'stop': None,
  'new_lat': None,
  'new_lng': None,
  'semantic_name': None,
  'foursquare_id': None,
  'original_name': None,
  'categories': None,
  'parent': None,
  'movement': None,
  'movement_prob': None,
  'city': None,
  'country': None,
  'new_timezone': None},
 '20000101_000145_000': {'Tags': 'text,wall,indoor',
  'OCR': None,
  'Caption': 'a room with a tv and a window',
  'Caption

In [14]:
json.dump(parsed, open('processed_data.json', 'w'))

In [15]:
len(parsed.keys())

725949